## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from joblib import dump, load

#from utils import precision_at_k, recall_at_k

## Data Loading

In [6]:
X_train_f = pd.read_csv("./output/X_train_f.csv") # 2010 - 2014, w. protected attributes
X_train_s = pd.read_csv("./output/X_train_s.csv") # 2010 - 2014, w/o protected attributes
y_train = pd.read_csv("./output/y_train.csv").iloc[:,0]

X_calib_f = pd.read_csv("./output/X_calib_f.csv") # 2015, w. protected attributes
X_calib_s = pd.read_csv("./output/X_calib_s.csv") # 2015, w/o protected attributes
y_calib = pd.read_csv("./output/y_calib.csv").iloc[:,0]

X_test_f = pd.read_csv("./output/X_test_f.csv")
X_test_s = pd.read_csv("./output/X_test_s.csv")
y_test = pd.read_csv("./output/y_test.csv").iloc[:,0]

## Correlation Analysis

In [7]:
# Computes the absolute value of the correlation matrix for the training features with protected attributes
corrM = X_train_f.corr().abs() # Corr matrix of X
corrM = corrM.unstack() # flatten
corrMo = corrM.sort_values(kind = "quicksort") # sort correlations
corrMo[corrMo < 1].tail(20) # Filters out the self-correlations (which equal 1) and prints the last 20 entries (lowest correlations)

ft_tot_dur_byage        ft_tot_dur                0.954161
ft_tot_dur              ft_tot_dur_byage          0.954161
maxbula.Missing.        lastjob_pt99999           0.954524
lastjob_pt99999         maxbula.Missing.          0.954524
seeking1_tot_dur_byage  seeking1_tot_dur          0.955420
seeking1_tot_dur        seeking1_tot_dur_byage    0.955420
lastjob_none            maxbula.Missing.          0.961120
maxbula.Missing.        lastjob_none              0.961120
lastjob_type99999       maxbula.Missing.          0.961120
maxbula.Missing.        tsince_lm_contact_cat5    0.961120
tsince_lm_contact_cat5  maxbula.Missing.          0.961120
maxbula.Missing.        lastjob_type99999         0.961120
                        lastjob_parallel99999     0.961120
lastjob_parallel99999   maxbula.Missing.          0.961120
emp1_total_dur          emp1_total_dur_byage      0.963495
emp1_total_dur_byage    emp1_total_dur            0.963495
secjob_tot_dur          secjob_tot_dur_byage      0.9658

## 01 Logit Regression (w. protected attributes)

In [12]:
glm1 = LogisticRegression(penalty = None, solver = 'lbfgs', max_iter = 1000)
glm1.fit(X_train_f, y_train)

/Users/julia/Desktop/CMA_Fairness/cma_f/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, penalty=None)

In [13]:
coefs1 = pd.DataFrame(X_train_f.columns, columns = ['var'])

In [14]:
coefs1['coef'] = pd.DataFrame(glm1.coef_).transpose()

In [16]:
dump(glm1, './models/glm1.joblib')

['./models/glm1.joblib']

## Predict

In [17]:
k75 = 0.75 # Top 75% 
k25 = 0.25 # Top 25% 
k10 = 0.1 # Top 10%

# Logit

In [18]:
glm1_p = glm1.predict_proba(X_test_f)[:,1] # glm1

In [19]:
threshold75 = np.sort(glm1_p)[::-1][int(k75*len(glm1_p))]
threshold25 = np.sort(glm1_p)[::-1][int(k25*len(glm1_p))]
threshold10 = np.sort(glm1_p)[::-1][int(k10*len(glm1_p))]

In [20]:
glm1_c1 = glm1_p.copy()
glm1_c1[glm1_c1 < threshold10] = 0
glm1_c1[glm1_c1 >= threshold10] = 1

In [21]:
glm1_c2 = glm1_p.copy()
glm1_c2[glm1_c2 < threshold25] = 0
glm1_c2[glm1_c2 >= threshold25] = 1

In [22]:
glm1_c3 = glm1_p.copy()
glm1_c3[(glm1_c3 <= threshold75) | (glm1_c3 >= threshold25)] = 0
glm1_c3[(glm1_c3 > threshold75) & (glm1_c3 < threshold25)] = 1

## Combine and save

In [23]:
preds_test = pd.concat([pd.DataFrame(np.array(y_test), columns = ['y_test']),
                         pd.DataFrame(glm1_p, columns = ['glm1_p']),
                         pd.DataFrame(glm1_c1, columns = ['glm1_c1']),
                         pd.DataFrame(glm1_c2, columns = ['glm1_c2']),
                         pd.DataFrame(glm1_c3, columns = ['glm1_c3'])],
                        axis = 1)

In [24]:
preds_test.to_csv('./output/preds_test.csv', index = False)